In [ ]:
from datetime import datetime, timedelta


today = datetime.now()
yesterday = today - timedelta(days=1)

# Lấy ngày dưới dạng string YYYY-MM-DD
yesterday_str = yesterday.strftime("%Y-%m-%d")

In [25]:
df_today

,symbol,date,open,close,pct_change,volume
0,YBM,2025-07-28,14600.0,15000.0,2.739726,26417.0
1,YEG,2025-07-28,13700.0,13900.0,1.459854,6374770.0
2,YTC,2025-07-28,45000.0,42300.0,-6.000000,5924.0
3,XPH,2025-07-28,14400.0,15600.0,8.333333,78240.0
4,YBC,2025-07-28,12500.0,12500.0,0.000000,517.0
...,...,...,...,...,...,...
1711,AAM,2025-07-28,7300.0,7380.0,1.095890,8268.0
1712,AAT,2025-07-28,3430.0,3420.0,-0.291545,98092.0
1713,AAH,2025-07-28,4200.0,4200.0,0.000000,4279165.0
1714,A32,2025-07-28,39500.0,39500.0,0.000000,0.0


In [24]:
import datetime
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

from src.config import RAW_DATA_DIR
from src.plots import get_stock_price
from src.tcbs_stock_data import TCBSStockData

today = datetime.datetime.now()
today_str = today.strftime("%Y-%m-%d")

stocks = pd.read_csv(RAW_DATA_DIR / "list_stock.csv")[["symbol"]]
stock_list = stocks["symbol"].tolist()


def fetch_stock_data(symbol):
    tcb = TCBSStockData()
    df = tcb.get_stock_data_by_date_range(
        ticker=symbol, start_date="2025-07-25", end_date="2025-07-25"
    )
    df["time"] = pd.to_datetime(df["time"])
    if df is not None and not df.empty:
        if not df.empty:
            open_price = df.iloc[0]["open"] if "open" in df else None
            close_price = df.iloc[0]["close"] if "close" in df else None
            volume = df.iloc[0]["volume"] if "volume" in df else None

            # Tính phần trăm thay đổi giá nếu đủ dữ liệu
            pct_change = None
            if open_price is not None and close_price is not None and open_price != 0:
                pct_change = 100 * (close_price - open_price) / open_price

            return {
                "symbol": symbol,
                "date": today_str,
                "open": open_price,
                "close": close_price,
                "pct_change": pct_change,
                "volume": volume,
            }
    # Nếu không có hoặc lỗi, trả về None
    return {
        "symbol": symbol,
        "date": today_str,
        "open": None,
        "close": None,
        "pct_change": None,
        "volume": None,
    }


num_threads = 16

results = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    future_to_symbol = {executor.submit(fetch_stock_data, s): s for s in stock_list}
    for future in as_completed(future_to_symbol):
        try:
            res = future.result()
            results.append(res)
        except Exception as e:
            symbol = future_to_symbol[future]
            results.append(
                {
                    "symbol": symbol,
                    "date": today_str,
                    "open": None,
                    "close": None,
                    "pct_change": None,
                    "volume": None,
                }
            )
            print(f"Lỗi lấy data cho {symbol}: {e}")

df_today = pd.DataFrame(results)

2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Lấy dữ liệu YTC từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Lấy dữ liệu YEG từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Bắt đầu lấy dữ liệu cho YTC từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Lấy dữ liệu YBM từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Bắt đầu lấy dữ liệu cho YEG từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Lấy dữ liệu YBC từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Lấy dữ liệu XPH từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,833 - tcbs_stock_data - INFO - Lấy dữ liệu XMP từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,849 - tcbs_stock_data - INFO - Lấy dữ liệu XMD từ 2025-07-25 đến 2025-07-25
2025-07-28 13:22:47,849 - tcbs_stock_data - INFO - Bắt đầu lấy dữ liệu cho YBM từ 2025-07-25 đến 2025-07-25
2025

Lỗi lấy data cho STG: 'time'


2025-07-28 13:23:03,091 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho STP: 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,107 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho STL: 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,107 - tcbs_stock_data - INFO - Đã lấy 250 điểm dữ liệu từ 2024-07-25
2025-07-28 13:23:03,108 - tcbs_stock_data - INFO - Đã lấy 250 điểm dữ liệu từ 2024-07-25
2025-07-28 13:23:03,109 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,110 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,111 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,114 - tcbs_stock_data - INFO - Lấy dữ liệu SRT từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,115 - tcbs_stock_data - INFO - Lấy dữ liệu SRF từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:03,115 - tcbs_stock_data - INFO - Lấy dữ 

Lỗi lấy data cho SKV: 'time'


2025-07-28 13:23:04,363 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho SMN: 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
2025-07-28 13:23:04,365 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho SMC: 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:04,366 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:04,369 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho SP2: 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:04,369 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07

...

2025-07-28 13:23:47,651 - tcbs_stock_data - INFO - Bắt đầu lấy dữ liệu cho CIP từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:47,604 - tcbs_stock_data - INFO - Lấy dữ liệu CII từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:47,620 - tcbs_stock_data - INFO - Đã lấy 250 điểm dữ liệu từ 2024-07-25
2025-07-28 13:23:47,620 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:47,620 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
2025-07-28 13:23:47,604 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho CMG: 1 điểm 

Lỗi lấy data cho BHI: 'time'
Lỗi lấy data cho BHA: 'time'


Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
2025-07-28 13:23:55,068 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho BHT: 0 điểm dữ liệu từ nan đến nan
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
2025-07-28 13:23:55,068 - tcbs_stock_data - WARNING - Không có dữ liệu cho BHT từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:55,068 - tcbs_stock_data - INFO - Lấy dữ liệu BCR từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:55,068 - tcbs_stock_data - INFO - Bắt đầu lấy dữ liệu cho BCR từ 2025-07-25 đến 2025-07-25
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/lat

Lỗi lấy data cho BDT: 'time'
Lỗi lấy data cho BCV: 'time'


2025-07-28 13:23:55,361 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:55,362 - tcbs_stock_data - INFO - Lấy dữ liệu BCB từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:55,362 - tcbs_stock_data - INFO - Bắt đầu lấy dữ liệu cho BCB từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:55,379 - tcbs_stock_data - INFO - Đã hoàn thành lấy dữ liệu cho BGE: 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
Unverified HTTPS request is being made to host 'proxy4.sgcoop.com.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
2025-07-28 13:23:55,379 - tcbs_stock_data - INFO - Đã lấy 250 điểm dữ liệu từ 2024-07-25
2025-07-28 13:23:55,379 - tcbs_stock_data - INFO - Đã lấy 250 điểm dữ liệu từ 2024-07-25
2025-07-28 13:23:55,379 - tcbs_stock_data - INFO - Lấy được 1 điểm dữ liệu từ 2025-07-25 đến 2025-07-25
2025-07-28 13:23:55,379 - tcbs_stock_data - INFO - Lấy dữ liệu BCA từ 2025-07